In [1]:
import requests
import json

import csv
import pandas as pd

import re

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
#DL from here https://spotipy.readthedocs.io/en/2.21.0/
#pip install spotipy --upgrade

from langdetect import detect
#DL from here https://pypi.org/project/langdetect/
#pip install langdetect

In [ ]:
#PARTITION THE DATASET

mypath = '../../ds2.csv'

In [76]:
#Your name?

#name = 'terrence'
#name = 'yenho'
#name = 'gabriela'
name = 'terrDesk'
#name = 'gabDesk'

In [2]:
if name == 'terrence':
    df = pd.read_csv(mypath, nrows=1000000)

elif name == 'yenho':
    df = pd.read_csv(mypath, skiprows=1000000, nrows=800080)
    
elif name = 'gabriela':
    df = pd.read_csv(mypath, skiprows=2000000, nrows=800080)
    
elif name == 'terrDesk':
    df = pd.read_csv(mypath, skiprows=3000000, nrows=800080)
    
elif name == 'gabDesk':
    df = pd.read_csv(mypath, skiprows=4000000, nrows=800080) #for Gabriela's desktop

In [3]:
#filter out nan values
df = df[df['lyrics'] == df['lyrics']]

In [4]:
#Filter out non english songs. This section will take a while.

def lang(row):
    lyrics = row['lyrics']
    
    try:
        lang = detect(lyrics)
    except:
        #print(row)
        #print(type(lyrics))
        lang = 'bs'
            
    return lang

df['lang'] = df.apply(lambda row : lang(row), axis = 1)

df = df[df['lang'] == 'en']

title       Pretty boy swag remix
tag                           rap
artist                 Gucci Mane
year                         2010
views                         408
features                       {}
lyrics                        ...
id                           1877
Name: 1889, dtype: object
<class 'str'>
title              Dear Diary
tag                       rap
artist                Cha Cha
year                     1999
views                     429
features    {"LaTocha Scott"}
lyrics                    ...
id                      13243
Name: 12611, dtype: object
<class 'str'>
title       Cut You Loose Freestyle
tag                             rap
artist                         Chop
year                           2011
views                            20
features                         {}
lyrics                          ...
id                            55689
Name: 40213, dtype: object
<class 'str'>
title       I am dreams of transcending
tag                                mis

title       RG Battle Round 3
tag                       rap
artist       Zen Master Griff
year                     2012
views                      81
features                   {}
lyrics                      .
id                      73354
Name: 68228, dtype: object
<class 'str'>
title       Wild Saturday Night
tag                         rap
artist              Donny Bravo
year                       2012
views                        43
features                     {}
lyrics                        [
id                        74072
Name: 68923, dtype: object
<class 'str'>
title       Ill be your inspiration   
tag                                rap
artist                    Ian Van Dahl
year                              2004
views                              112
features                            {}
lyrics                             ???
id                               75580
Name: 70353, dtype: object
<class 'str'>
title       Southside savage
tag                      rap
artist     

title       Im Single But I Still Love You
tag                                    rap
artist                               Pewee
year                                  2012
views                                  147
features                                {}
lyrics                                   .
id                                   90906
Name: 84521, dtype: object
<class 'str'>
title       Butterflies and Toast
tag                           rap
artist             Brady (Rapper)
year                         2012
views                          64
features                   {Alec}
lyrics                          [
id                          92559
Name: 85987, dtype: object
<class 'str'>
title       I dont love em
tag                    rap
artist         Gangsta Boo
year                  2012
views                  177
features                {}
lyrics                   .
id                   93752
Name: 87011, dtype: object
<class 'str'>
title       Goldie Remix
tag                 

title                               Hustlin
tag                                     rap
artist                         Joey Zadjino
year                                   2012
views                                    51
features    {"Andrew Johnathon Eggebrecht"}
lyrics                                    4
id                                   110288
Name: 102486, dtype: object
<class 'str'>
title       Top Of The World
tag                      rap
artist            Nick Proko
year                    2013
views                     48
features                  {}
lyrics                   ...
id                    110319
Name: 102517, dtype: object
<class 'str'>
title       Rap sucks
tag               rap
artist       Reh Dogg
year             2013
views              49
features           {}
lyrics            ...
id             112111
Name: 104319, dtype: object
<class 'str'>
title       Mutlu Ol Yeter
tag                    rap
artist             D-Peace
year                  2013
views 

title                   WTF
tag                     rap
artist      Pablo & Tommy O
year                   2013
views                    30
features                 {}
lyrics                    [
id                   150556
Name: 140791, dtype: object
<class 'str'>
title       Minter Monderland
tag                       rap
artist                    R89
year                     2013
views                      16
features                   {}
lyrics                      _
id                     165816
Name: 141934, dtype: object
<class 'str'>
title       Sputnik 69
tag                rap
artist             R89
year              2013
views               34
features            {}
lyrics               _
id              165817
Name: 141935, dtype: object
<class 'str'>
title                                          Foreva Huslin
tag                                                      rap
artist                                             Cory Gunz
year                                       

...

title       TheRapXChain
tag                  rap
artist       MC_CrossDK7
year                2015
views                 28
features              {}
lyrics               :((
id                704782
Name: 435413, dtype: object
<class 'str'>
title                           Cause She Love Me
tag                                           rap
artist                            Deuce & $ilence
year                                         2015
views                                          16
features    {"Deuce & \\$ilence","Tyree Bridges"}
lyrics                                          (
id                                         706189
Name: 436358, dtype: object
<class 'str'>
title       Season 2 Episode 15 Music
tag                              misc
artist      The Originals (TV Series)
year                             2015
views                             348
features                           {}
lyrics                              .
id                             710845
Name: 437394

title           R.
tag            rap
artist      Jeezus
year          2015
views          127
features        {}
lyrics         ...
id          717070
Name: 444224, dtype: object
<class 'str'>
title       Fools Gold
tag                rap
artist            Evan
year              2015
views               72
features            {}
lyrics               .
id              717261
Name: 444411, dtype: object
<class 'str'>
title              Isabella 1849
tag                         misc
artist      John Everett Millais
year                        1849
views                         37
features                      {}
lyrics                    \n\n\n
id                        717760
Name: 444813, dtype: object
<class 'str'>
title           Les pierres dinfinité
tag                              misc
artist      Marvel Cinematic Universe
year                             2015
views                            3065
features                           {}
lyrics                         \n\n\n
id      

title       3 Hit Combo
tag                 rap
artist             41 F
year               2015
views                30
features      {"41° F"}
lyrics                (
id               736004
Name: 457106, dtype: object
<class 'str'>
title              Писька
tag                   rap
artist      Youyounaxsuka
year                 2015
views                  92
features               {}
lyrics                ...
id                 737322
Name: 457784, dtype: object
<class 'str'>
title       Rich Man Poor Man
tag                       rap
artist           Car.tell.one
year                     2014
views                      60
features                   {}
lyrics                      .
id                     737480
Name: 457908, dtype: object
<class 'str'>
title       Разве бэтмен?
tag                   rap
artist        MsVolosatka
year                 2015
views                  17
features               {}
lyrics                ...
id                 738844
Name: 458857, dtype: objec

title           Dizzy
tag               pop
artist      A.R. Kane
year             1988
views             139
features           {}
lyrics              .
id             821673
Name: 534321, dtype: object
<class 'str'>
title                       Floresta
tag                              pop
artist                  Jon Anderson
year                            1994
views                             80
features    {"Birdsinging & Ronata"}
lyrics                           [?]
id                            822846
Name: 535493, dtype: object
<class 'str'>
title                 Tantrum
tag                       rap
artist      Tha lynch project
year                     2015
views                       7
features                   {}
lyrics                      -
id                     824235
Name: 536582, dtype: object
<class 'str'>
title       Wheres Tha Antidote
tag                         rap
artist        Tha lynch project
year                       2015
views                        43
fe

title          Stahhle
tag                rap
artist      Dratchface
year              2015
views               12
features            {}
lyrics               ?
id             1260510
Name: 954965, dtype: object
<class 'str'>


In [5]:
#Process lyrics

def lyricEdit(row):
    lyrics = row['lyrics']

    lyrics = re.sub("[\[].*?[\]]", "", lyrics)

    lyrics = re.sub(r'\n\s*\n', '\n', lyrics)
    
    lyrics = lyrics.split('\n')

    if lyrics[0] == '':
            lyrics = lyrics[1:]
            
    return lyrics

df['lyrics'] = df.apply(lambda row : lyricEdit(row), axis = 1)

In [6]:
df.shape #from a million, i was able to still have 786k after all this filtering.

(786528, 9)

In [77]:
def swapKeys(user, idx):
    while True:
        cidLst = ["b7b309716161449da86485a6348a346e", 
                  "6c401ae9ee5f4b5286249499b2633ead",
                  "758e3d5b94a14815ae6d336d7e37b209",
                  "0da9f55d56884f99869265490b334326"]

        if user == "terrence":
            cidLst.append("715ae75406e24c24a3641929260ca343")
        elif user == "yenho":
            cidLst.append("e3fc9e837db6490184d4095450c3508f")
        elif user == "gabriela":
            cidLst.append("7eabd16573dc4a23bbb90b2e6578ba9d")
        elif user == "terrDesk":
            cidLst.append("4b387b6ec9d6446db9035cb56d744fec")
        elif user == 'gabDesk':
            cidLst = cidLst[::-1]
            cidLst.append("0daf9f613d0946d780ebfb062c0d8ad7")

        secretLst = ["8d8dea83579b4501b0526f45c671c94b", 
                     "ef9cfc951ddd4913986588b761a1a55d",
                     "5448a297b3ba4ebd907eb2d2657ea15b",
                     "be709045ba894650952c1733af4d201e"]

        if user == "terrence":
            secretLst.append("e60a686258c646b7a4d3f9636b3235db")
        elif user == "yenho":
            secretLst.append("493edd8594fa4509b1ea52c7a1faae5c")
        elif user == "gabriela":
            secretLst.append("732827c5c1a2499a853c12ec73ca5264")
        elif user == "terrDesk":
            secretLst.append("dbeee0427c8f41dd99f5703b157ee252")
        elif user == 'gabDesk':
            secretLst = secretLst[::-1]
            secretLst.append("3b3245dde0404a6b87906235d9a9971a")

        cid = cidLst[idx]
        secret = secretLst[idx]

        idx += 1

        if idx == len(cidLst):
            idx = 0

        AUTH_URL = "https://accounts.spotify.com/api/token"
        
        try:
            auth_response = requests.post(AUTH_URL, {
                'grant_type': 'client_credentials',
                'client_id': cid,
                'client_secret': secret,
            })
        except:
            print('\nHaving issues connecting... trying again.')
            continue

        try:
            auth_response_data = auth_response.json()
        except:
            print('\nHaving issues connecting... trying again.')
            continue

        access_token = auth_response_data['access_token']

        headers = {
            'Authorization': 'Bearer {token}'.format(token=access_token)
        }

        return headers, idx


In [85]:
def getUri(row, idx):
    headers, _ = swapKeys(name, idx)
    
    artistName = row['artist']
    trackName = row['title']

    artistName = re.sub("[\(\[].*?[\)\]]", "", artistName)
    trackName = re.sub("[\(\[].*?[\)\]]", "", trackName)

    trackName2 = trackName.replace(' ', '%20')
    artistName2 = artistName.replace(' ', '%20')
    
    trackName = trackName.upper()
    trackName = trackName.replace("'", "")
    trackName = trackName.replace("&", "")
    trackName = trackName.replace(" ", "")
    
    artistName = artistName.upper()
    artistName = artistName.replace("'", "")
    artistName = artistName.replace("&", "")
    artistName = artistName.replace(" ", "")
    
    attempts = 0
    trying = True
    
    while(trying):
        BASE_URL = 'https://api.spotify.com/v1/'
        r = requests.get(BASE_URL + 'search?q=' + artistName2 + '%20track:' + trackName2 + '&type=track', headers=headers)
        
        while(True):
            try:
                r = r.json()
                break
            except:
                #if str(r)[-5:-2] == "429":
                print("\tIt seems rate limit is reached, swapping keys")
                headers, idx = swapKeys(name, idx)
                r = requests.get(BASE_URL + 'search?q=' + artistName2 + '%20track:' + trackName2 + '&type=track', headers=headers)
    
        if 'tracks' in r:
            break
        else:
            print('\nTracks not found in r, trying again and swapping gang')
            attempts += 1
            headers, idx = swapKeys(name, idx)
            
            if attempts == 10:
                trying = False
            
            print('\t',r)
            
    if trying == False:
        print('\tcouldnt get a search. skipping for now',artistName,'|',trackName)
        return '', idx

    if r['tracks']['items'] == []:
        print('\tno search results found:',artistName,'|',trackName)
        return '', idx

    for item in r['tracks']['items']:
        editedName = item['name'].upper()
        editedName = editedName.replace("'", "")
        editedName = editedName.replace("&", "")
        editedName = editedName.replace(" ", "")
        
        editedArt = item['artists'][0]['name'].upper()
        editedArt = editedArt.replace("'", "")
        editedArt = editedArt.replace("&", "")
        editedArt = editedArt.replace(" ", "")
        
        trackCorr = (editedName == trackName) or (editedName in trackName) or (trackName in editedName)
        artCorr = (editedArt == artistName) or (editedArt in artistName) or (artistName in editedArt)
        
        if (trackCorr or artCorr):
            print('\tgottem:',artistName,'|',trackName)
            return item['uri'], idx

    print('\tnone of the items matched:',artistName,'|',trackName)
    return '', idx
        

In [79]:
#UNCOMMENT THIS THEN RUN IT. AFTER YOU RUN IT, COMMENT IT AGAIN
#BECAUSE YOU CANNOT RISK RUNNING THIS CELL AGAIN IF YOURE DEEP INTO RECORDING URIS (IT WILL RESET THE CSV AND LIST)
'''
uriLst = []

realUris = 0

#writing URIs to a csv just to save them somewhere :)
with open('uri.csv', 'w', newline='') as fCsv:
    writer = csv.writer(fCsv)

    header = ['uri']
    writer.writerow(header)

'''

"\nuriLst = []\n\nrealUris = 0\n\n#writing URIs to a csv just to save them somewhere :)\nwith open('uri.csv', 'w', newline='') as fCsv:\n    writer = csv.writer(fCsv)\n\n    header = ['uri']\n    writer.writerow(header)\n\n"

In [86]:
idx = 2
with open('uri.csv', 'a', newline='') as fCsv:
    writer = csv.writer(fCsv)
    
    for index, row in df.iloc[len(uriLst):].iterrows():
        if realUris >= 200020:
            break
            
        print('\nOn row:',len(uriLst), '| # of real URIs obtained:',realUris)
        uri, idx = getUri(row, idx)
        
        if uri != '':
            realUris += 1
        
        uriLst.append(uri)
        
        writer.writerow([uri])
#'''


On row: 52240 | # of real URIs obtained: 33861

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}
	It seems rate limit is reached, swapping keys

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}

Tracks not found in r, trying again and swapping gang
	 {'error': {'status': 400, 'message': 'No search query'}}
	It seems rate limit is reached, swapping keys

Tracks not found in r, trying again and swapping gang
	 {'e

	gottem: DRAKE | ONMYWAY

On row: 52317 | # of real URIs obtained: 33897
	gottem: JAMIEFOXX | BESTNIGHTOFMYLIFE

On row: 52318 | # of real URIs obtained: 33898
	no search results found: CZARIFICK | CALLITFRIENDO

On row: 52319 | # of real URIs obtained: 33898
	no search results found: TUYO3 | THEINTRORAP

On row: 52320 | # of real URIs obtained: 33898
	gottem: LOWKEY | HANDONYOURGUN

On row: 52321 | # of real URIs obtained: 33899
	no search results found: ROACH | YONKERSREMIX

On row: 52322 | # of real URIs obtained: 33899
	no search results found: ZSAZSAGABOR | EXHIBITZ

On row: 52323 | # of real URIs obtained: 33899
	gottem: MACMILLER | ALLTHIS

On row: 52324 | # of real URIs obtained: 33900
	gottem: LADABOOMMAN | UPINHERE

On row: 52325 | # of real URIs obtained: 33901
	gottem: MACMILLER | ILOVELIFETHANKYOU

On row: 52326 | # of real URIs obtained: 33902
	gottem: GUCCIMANEWAKAFLOCKAFLAME | PACMAN

On row: 52327 | # of real URIs obtained: 33903
	gottem: AZEALIABANKS | 212

On row: 52

	gottem: WAKAFLOCKAFLAME | RAYRAY

On row: 52408 | # of real URIs obtained: 33942
	no search results found: NBS | NOBULLSHIT

On row: 52409 | # of real URIs obtained: 33942
	no search results found: ALBEBACK | MIRAMIRAVENAQUI

On row: 52410 | # of real URIs obtained: 33942
	gottem: XV | BLAMEYOURSELF

On row: 52411 | # of real URIs obtained: 33943
	gottem: TALIBKWELI | BEAUTIFUL

On row: 52412 | # of real URIs obtained: 33944
	gottem: QUASIMOTO | SEASONSCHANGE

On row: 52413 | # of real URIs obtained: 33945
	gottem: STRONGARMSTEADY | CHITTLINSPEPSI

On row: 52414 | # of real URIs obtained: 33946
	no search results found: QUELLKEITH | THOUSANDDOLLA

On row: 52415 | # of real URIs obtained: 33946
	no search results found: 50CENT | SEEMEBLEEDING

On row: 52416 | # of real URIs obtained: 33946
	gottem: TREYSONGZ | SAYAAH

On row: 52417 | # of real URIs obtained: 33947
	gottem: STATIKSELEKTAH | GROUPIELOVE

On row: 52418 | # of real URIs obtained: 33948
	no search results found: A$APROCKY |

	no search results found: PACDIV | WAVESATTHEPROM

On row: 52498 | # of real URIs obtained: 33987
	no search results found: DRAKE | IMONMYWAY

On row: 52499 | # of real URIs obtained: 33987
	no search results found: DRAKE | POPROSÉ

On row: 52500 | # of real URIs obtained: 33987
	no search results found: THEMADCONDUCTOR | SOULLESSEXPERIENCE

On row: 52501 | # of real URIs obtained: 33987
	no search results found: KAHLIL | SKYAIMING

On row: 52502 | # of real URIs obtained: 33987
	gottem: PHONTE | EVERYTHINGISFALLINGDOWN

On row: 52503 | # of real URIs obtained: 33988
	no search results found: EINSTEIN | ORIENTATIONINTRO

On row: 52504 | # of real URIs obtained: 33988
	no search results found: BIRDMAN | YOUTOOFINE

On row: 52505 | # of real URIs obtained: 33988
	no search results found: ADE | COLLEGEGIRL

On row: 52506 | # of real URIs obtained: 33988
	none of the items matched: BLUEXILE | THENARROWPATH

On row: 52507 | # of real URIs obtained: 33988
	no search results found: DRAKE | SE

KeyboardInterrupt: 

In [ ]:
df['uri'] = uriLst
df = df[df['uri'] != '']

df.shape

In [14]:
if name == 'terrence':
    df.to_csv('musicData.csv', index=False)
    
elif name == 'yenho':
    df.to_csv('musicData2.csv', index=False)
    
elif name == 'gabriela':
    df.to_csv('musicData3.csv', index=False)
    
elif name == 'terrDesk':
    df.to_csv('musicData4.csv', index=False)
    
elif name == 'gabDesk':
    df.to_csv('musicData5.csv', index=False)